In [1]:
import pandas as pd
import numpy as np
import stellargraph

data = pd.read_csv('BollyBAIT.csv')
data.columns

Index(['Title', 'Video ID', 'Video URL', 'Channel Title', 'Misleading Video',
       'False Promises ', 'Exaggerated Video', 'Spam Content ',
       'Exploits Curiosity Gap :', 'Label'],
      dtype='object')

In [11]:
import os
import pickle
import stellargraph
labels = []
graphs=[]
for ind in data.index:
    name_tag = data['Video ID'][ind]
    #youtube_video_url = "https://www.youtube.com/watch?v=" + name_tag
    #folder_path = "C:/Users/minij/Downloads/Clickbait Detection/Keyframes/" + name_tag + "/"
    #directory="C:/Users/DELL/Desktop/clickbaitmajor/bollybait/"+name_tag
    if (os.path.exists("/Users/peyamowar/Downloads/clickbait_youtube/graph cnn bollybait/graph cnn/"+name_tag)==False):
        print(name_tag)
        continue
    file1= open("/Users/peyamowar/Downloads/clickbait_youtube/graph cnn bollybait/graph cnn/"+name_tag,'rb')
    #+".txt"
    source=[]
    target=[]
    edgewt=[]
    node_feature=[]
    edgemat=[]
    while True:
        try:
        #reading the oject from file
            employee = pickle.load(file1)
            source = employee['source']
            target = employee['target']
            edgewt = employee['weight']
            node_feature = employee['nodes']
            edgemat = employee['edges']
           
        except EOFError:
            break
    file1.close()
    video_name = name_tag
    '''
    if (os.path.exists(folder_path+"/keyframes")==False):
        print(name_tag)
        continue
    if (os.path.exists(folder_path+"/keyframeresnetfeatures.csv")==False):
        print(name_tag)
        continue
    path, dirs, files = next(os.walk(folder_path+"/keyframes"))
    file_count = len(files)
    if (file_count==0):
        print(name_tag)
        continue
    edgematlen=file_count
    source=[i for i in range(1,edgematlen+1)]
    target=[0 for i in range(1,edgematlen+1)]
    node_features=[]
    directorykeyframes = directory + "/keyframes"
    
    thumbnail = directory + "/" + name_tag + "_thumbnail.jpg"
    list_dir = os.listdir(folder_path+"keyframes")
    list_dir = [f.lower() for f in list_dir]
    list_dir = sorted(list_dir)
    edgewt=np.empty((0,1), float)
    #dgewt.append(imagesim(thumbnail, thumbnail))
    co=0
    
    for file in list_dir:
        if file != '.ds_store':
            imgpath = folder_path+"keyframes"+"/"+file 
            #yen = imagesim(thumbnail, imgpath)['output']['distance']
            #yen=imagesimilarity(thumbnail, imgpath)
            #yen=sift_sim(thumbnail, imgpath)
            yen=imagesimval(thumbnail, imgpath)
            edgewt=np.append(edgewt, yen)
            co=co+1
    if (co!=edgematlen):
        print(name_tag)
        continue
    dataframe = pd.read_csv(folder_path + "keyframeresnetfeatures.csv")
    mat1=dataframe.to_numpy()
    '''
    print(type(node_feature))
    print(node_feature)
    node_features=pd.DataFrame(node_feature)
    #print(edgewt.shape)
    #edgewt.reshape(edgematlen,1)
    #print(mat1.shape)
    #print(edgewt.shape)
    #print(video_name)
    #print(node_features)
    #print(edgewt)
    print(ind)
    labels.append(data['Label'][ind])
    edgemat=pd.DataFrame({"source": source, "target": target, "weight": edgewt})
    graph = stellargraph.StellarGraph(nodes = node_features, edges=edgemat)
    graphs.append(graph)

<class 'pandas.core.frame.DataFrame'>


AttributeError: 'NoneType' object has no attribute 'axes'

In [5]:
print(graphs)

[]


In [13]:
node_feature

AttributeError: 'NoneType' object has no attribute 'axes'

In [ ]:
import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import GCNSupervisedGraphClassification
from stellargraph import StellarGraph

from stellargraph import datasets

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarGraph

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf

df = pd.DataFrame(labels, columns=['Label'])

graph_labels = pd.get_dummies(df['Label'])

In [ ]:
graph_labels

In [ ]:
generator = PaddedGraphGenerator(graphs=graphs)

In [ ]:
k = 35  # the number of rows for the output tensor
layer_sizes = [64, 64, 64, 1]

dgcnn_model = DeepGraphCNN(
    layer_sizes=layer_sizes,
    activations=["tanh", "tanh", "tanh", "tanh"],
    k=k,
    bias=False,
    generator=generator,
)
x_inp, x_out = dgcnn_model.in_out_tensors()

In [ ]:
x_out = Conv1D(filters=32, kernel_size=sum(layer_sizes), strides=sum(layer_sizes))(x_out)
x_out = MaxPool1D(pool_size=2)(x_out)

x_out = Conv1D(filters=64, kernel_size=5, strides=1)(x_out)

x_out = Flatten()(x_out)

x_out = Dense(units=128, activation="relu", name="layer5")(x_out)
x_out = Dropout(rate=0.5)(x_out)
x_out = Dense(units=64, activation="relu", name="layer7")(x_out)
x_out = Dropout(rate=0.5)(x_out)
x_out = Dense(units=16, activation="relu", name="layer9")(x_out)
x_out = Dropout(rate=0.5)(x_out)

predictions = Dense(units=1, activation="sigmoid")(x_out)

In [ ]:
model = Model(inputs=x_inp, outputs=predictions)

model.compile(
    optimizer=Adam(lr=0.0001), loss=binary_crossentropy, metrics=["acc"],
)

In [ ]:
train_graphs, test_graphs = model_selection.train_test_split(
    graph_labels, train_size=0.8, test_size=None, stratify=graph_labels,
)

In [ ]:
gen = PaddedGraphGenerator(graphs=graphs)

train_gen = gen.flow(
    list(train_graphs.index - 1),
    targets=train_graphs.values,
    batch_size=1,
    symmetric_normalization=False,
)

test_gen = gen.flow(
    list(test_graphs.index - 1),
    targets=test_graphs.values,
    batch_size=1,
    symmetric_normalization=False,
)

In [ ]:
epochs = 10
history = model.fit(
    train_gen, epochs=epochs, verbose=1, validation_data=test_gen, shuffle=True,
)

In [ ]:
test_metrics = model.evaluate(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [ ]:
predictions = model.predict(train_gen)
# round predictions 
rounded = [round(x[0]) for x in predictions]
predictions

In [ ]:
extractor = Model(model.inputs, model.get_layer('layer7').output)
features = extractor.predict(train_gen)
features